In [4]:
import plotly.graph_objects as go
import numpy as np
import random
from IPython.display import display
from ipywidgets import Button, IntSlider, HBox, VBox, Label

# Configuración inicial
num_particles = 50
particle_size = 6
target_size = 12
measurement_size = 40
particles = []
weights = []
is_running = False

# Inicialización de partículas, objetivo y medición
def initialize_particles():
    global particles, weights
    particles = np.random.rand(num_particles, 2) * 600  # Posiciones iniciales
    weights = np.ones(num_particles) / num_particles

target = np.array([300.0, 200.0])
measurement = np.array([300.0, 200.0])

initialize_particles()

# Gráfico inicial
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=particles[:, 0],
    y=particles[:, 1],
    mode='markers',
    marker=dict(size=particle_size, color='blue', opacity=0.5),
    name='Particles'
))

fig.add_trace(go.Scatter(
    x=[target[0]],
    y=[target[1]],
    mode='markers',
    marker=dict(size=target_size, color='red'),
    name='Target'
))

fig.add_trace(go.Scatter(
    x=[measurement[0]],
    y=[measurement[1]],
    mode='markers',
    marker=dict(size=measurement_size, color='gray', symbol='circle', opacity=0.3),
    name='Measurement'
))

fig.update_layout(
    width=600,
    height=400,
    xaxis=dict(range=[0, 600]),
    yaxis=dict(range=[0, 400]),
    showlegend=True,
    title='Particle Filter Simulation'
)

# Funciones para actualizar el gráfico
def move_particles():
    global particles
    noise = (np.random.rand(num_particles, 2) - 0.5) * 10
    particles += noise
    particles[:, 0] = np.clip(particles[:, 0], 0, 600)
    particles[:, 1] = np.clip(particles[:, 1], 0, 400)

def update_target_and_measurement():
    global target, measurement
    target += (np.random.rand(2) - 0.5) * 5
    measurement = target + (np.random.rand(2) - 0.5) * 20
    target = np.clip(target, [0, 0], [600, 400])
    measurement = np.clip(measurement, [0, 0], [600, 400])

def update_weights():
    global weights
    distances = np.linalg.norm(particles - measurement, axis=1)
    weights = 1 / (1 + distances)
    weights /= weights.sum()

def resample_particles():
    global particles
    indices = np.random.choice(range(num_particles), size=num_particles, p=weights)
    particles = particles[indices] + (np.random.rand(num_particles, 2) - 0.5) * 10

# Función de animación
def animate():
    global is_running
    if not is_running:
        return
    move_particles()
    update_target_and_measurement()
    update_weights()
    if random.random() < 0.1:
        resample_particles()
    update_plot()
    fig.show()

# Actualizar el gráfico
def update_plot():
    fig.data[0].x = particles[:, 0]
    fig.data[0].y = particles[:, 1]
    fig.data[1].x = [target[0]]
    fig.data[1].y = [target[1]]
    fig.data[2].x = [measurement[0]]
    fig.data[2].y = [measurement[1]]

# Controles
start_button = Button(description="Start")
reset_button = Button(description="Reset")
speed_slider = IntSlider(value=50, min=1, max=100, description="Speed:")
speed_label = Label("50%")

def start_animation(_):
    global is_running
    is_running = True
    animate()

def reset_animation(_):
    global is_running
    is_running = False
    initialize_particles()
    update_plot()
    fig.show()

start_button.on_click(start_animation)
reset_button.on_click(reset_animation)
# Función para actualizar la etiqueta de velocidad
def update_speed_label(change):
    speed_label.value = f"{change['new']}%"

# Cambiar observador
speed_slider.observe(update_speed_label, names='value')


# Mostrar los controles
display(VBox([HBox([start_button, reset_button]), speed_slider, speed_label]))

# Mostrar el gráfico inicial
fig.show()